# Domain pruning in holocleansession 
In this notebook we will show the mechanism of domain pruning and how it works. To start we import the necessary modules 

In [1]:
import sys
import os
sys.path.append('../')
from holoclean import holocleansession , dataset , dataengine

After import file we have to indicate the driver file path for mysql connector and initialize the holocleansession with it.

In [2]:
my_path = os.path.abspath(os.path.dirname('__file__'))
path = os.path.join(my_path, "../holoclean/mysql-connector-java-5.1.44-bin.jar")
holoclean_se=holocleansession.HolocleanSession(path)

We have to create dataset object to create specific object that track the project tables in the database and after that we ingest the csv to spark and database.

In [3]:
ds=dataset.Dataset()  
d=dataengine.Dataengine("metadb-config.txt",'datadb-config.txt',ds,holoclean_se.return_sqlcontext())
holoclean_se.set_dataengine(d)
df=d.ingest_spark('10.csv',holoclean_se.returnspark_session())

index


In here based on the denial constraints we divide the cells into two sets cells that we know their values are clean and cells that have some violations.

In [4]:
dcCode=['t1&t2&EQ(t1.city,t2.city)&EQ(t1.temp,t2.temp)&IQ(t1.tempType,t2.tempType)']

dk_cells,clean_cells=holoclean_se._error_detection(dcCode,df)

ind
ind


In here we try to prune the domain the default value for threshold is 0.5

In [5]:
can=holoclean_se._domain_prunnig()
can.show(can.count())

attr
+--------+-----------+
|    attr|      value|
+--------+-----------+
|    date|   18000101|
|    date|   18000102|
|    city|EZE00100082|
|    city|ITE00100554|
|tempType|       TMAX|
|tempType|       TMIN|
|    temp|       -135|
+--------+-----------+



In here we can call the table from mysql database too

In [6]:
d_in_db=d.get_table_spark("D")
d_in_db.show(d_in_db.count())

+--------+-----------+
|    attr|      value|
+--------+-----------+
|    date|   18000101|
|tempType|       TMIN|
|    temp|       -135|
|    city|ITE00100554|
|tempType|       TMAX|
|    date|   18000102|
|    city|EZE00100082|
+--------+-----------+



Next challenge is change the threshold value to zero and see it return all values or not

In [7]:
can2=holoclean_se._domain_prunnig(new_threshold = 0)
can2.show(can2.count())

attr
+--------+-----------+
|    attr|      value|
+--------+-----------+
|    date|   18000101|
|    date|   18000102|
|    city|EZE00100082|
|    city|GM000010962|
|    city|ITE00100554|
|tempType|       PRCP|
|tempType|       TMAX|
|tempType|       TMIN|
|    temp|       -125|
|    temp|       -135|
|    temp|       -148|
|    temp|        -44|
|    temp|        -60|
|    temp|        -75|
|    temp|        -86|
|    temp|          0|
+--------+-----------+



Again we check that the new table has been substitute in database 

In [8]:
d2_in_db=d.get_table_spark("D")
d2_in_db.show(d2_in_db.count())

+--------+-----------+
|    attr|      value|
+--------+-----------+
|    temp|        -60|
|    temp|        -75|
|    temp|        -86|
|    city|ITE00100554|
|    temp|       -125|
|tempType|       PRCP|
|tempType|       TMAX|
|tempType|       TMIN|
|    temp|          0|
|    date|   18000101|
|    temp|       -135|
|    temp|       -148|
|    date|   18000102|
|    temp|        -44|
|    city|EZE00100082|
|    city|GM000010962|
+--------+-----------+

